In [1]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparametrization_space as kernels_repar_space
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [2]:
space: List[str] = ['4', '4']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [6]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    #hour_end_index = (day_index + 1) * 8
    hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=ord_mm,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([1120, 4])


Hyper parameters

In [49]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
lr = 0.01
step = 100
gamma_par = 0.5
epochs = 400

Adams Optimization Full batch

In [50]:
# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    
    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # [1] phi2 = 1 / range_lon
    init_phi1 = init_sigmasq * init_phi2            # [0] phi1 = sigmasq * phi2 = sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # [2] phi3 = (range_lon / range_lat)^2
    # phi4 is removed
    
    # Initialize the parameter list by taking the log(),
    # because the covariance function uses torch.exp().
    # 
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 💥 REVISED: 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [3] log(nugget)
    ]

    # --- Define learning rates and parameter groups ---
    lr_slow, lr_fast = 0.005, 0.01 
    
    # 💥 REVISED: All 4 parameters were in your original "fast" group
    # The "slow" group (advection/beta) is now empty.
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    # (This assumes 'kernels_new' has the 'fit_vecchia_adams' class)

    model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- Call the Correct Optimizer Method ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            step_size=step, 
            gamma=gamma_par
        )

    # --- Call the Correct Fit Method ---
    # 💥 REVISED: Pass the new 4-parameter SPATIAL covariance function
    out, epoch_ran = model_instance.fit_vecc_scheduler_fullbatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, # Pass the new spatial-only method
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1092, smooth: 0.5
mm_cond_number: 8,
initial parameters: 
 [tensor([3.0647], dtype=torch.float64, requires_grad=True), tensor([0.3567], dtype=torch.float64, requires_grad=True), tensor([0.1177], dtype=torch.float64, requires_grad=True), tensor([0.4055], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 1805.816426 ---
  Param 0: Value=3.0647, Grad=-103.49208029611192
  Param 1: Value=0.3567, Grad=1.9131134813683068
  Param 2: Value=0.1177, Grad=-58.43307834287484
  Param 3: Value=0.4055, Grad=-64.86826012260832
  Max Abs Grad: 1.034921e+02
------------------------------
--- Epoch 11 / Loss: 1788.689645 ---
  Param 0: Value=3.1619, Grad=-48.75344694035992
  Param 1: Value=0.3596, Grad=-3.626484491954619
  Param 2: Value=0.2162, Grad=-38.080729928796636
  Param 3: Value=0.5036, Grad=-38.54812886082654
  Max Abs Grad: 4.875345e+01
------------------------------
--- Epoch 21 / Loss: 1780.617411 ---
  Param 0: Value=3.2417, 

Adams Optimizatin using Minibatch

For apple to apple comparison, I set nheads=1 to minimize the effect of nheads.

In [51]:
# Assume global variables are defined elsewhere:
# daily_hourly_maps, daily_aggregated_tensors, nns_map, kernels_repar_space, device_str
import time 
from pathlib import Path
import numpy as np
import torch
from typing import List, Dict, Any, Tuple, Optional


# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    # Assuming daily_hourly_maps, daily_aggregated_tensors, and nns_map are loaded successfully.
    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                
    init_phi1 = init_sigmasq * init_phi2            
    init_phi3 = (init_range_lon / init_range_lat)**2 
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list (using log transform)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [3] log(nugget)
    ]

    # --- Define learning rates and parameter groups ---
    lr_fast = lr # Use the global lr
    fast_indices = [0, 1, 2, 3] # All parameters
    
    param_groups = [
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    # --- Print Job Info ---
    # Calculated based on assumed 113x158 degree domain and 4x4 resolution
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[1]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per hour: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number}, batch_size (time): {batch_size_time}')
    print(f'Initial parameters (log): \n {params_list}')
            
    # --- Instantiate the Correct Class ---
    # Assuming kernels_repar_space is defined and holds the class
    model_instance = kernels_repar_space.fit_vecchia_adams_minibatch(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- Call the Correct Optimizer Method ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # --- Call the NEW Fit Method with Mini-Batching ---
    out, epoch_ran = model_instance.fit_vecc_scheduler_minibatch(
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_SPATIAL_log_reparam, 
            epochs=epochs,
            batch_size=batch_size_time 
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"\nDay {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per hour: 1092, smooth: 0.5
mm_cond_number: 8, batch_size (time): 16
Initial parameters (log): 
 [tensor([3.0647], dtype=torch.float64, requires_grad=True), tensor([0.3567], dtype=torch.float64, requires_grad=True), tensor([0.1177], dtype=torch.float64, requires_grad=True), tensor([0.4055], dtype=torch.float64, requires_grad=True)]
--- Epoch 001 / Avg NLL per Obs: 1.612336 ---
  Param 0: Value=3.074725, Grad=-9.2404e-02
  Param 1: Value=0.346675, Grad=1.7081e-03
  Param 2: Value=0.127681, Grad=-5.2172e-02
  Param 3: Value=0.415465, Grad=-5.7918e-02
  Max Abs Grad: 9.240364e-02
------------------------------
--- Epoch 002 / Avg NLL per Obs: 1.610350 ---
  Param 0: Value=3.084705, Grad=-8.6710e-02
  Param 1: Value=0.337409, Grad=8.1518e-04
  Param 2: Value=0.137670, Grad=-5.0196e-02
  Param 3: Value=0.425451, Grad=-5.5250e-02
  Max Abs Grad: 8.670953e-02
------------------------------
--- Epoch 003 / Avg NLL per Obs: 1.608479 ---
  Param 0: V

L-BFGS Optimization

In [52]:
# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Correct Parameter Initialization (SPATIAL-ONLY) ---
    
    # Define initial *model* parameters
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # [1] phi2 = 1 / range_lon
    init_phi1 = init_sigmasq * init_phi2            # [0] phi1 = sigmasq * phi2 = sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # [2] phi3 = (range_lon / range_lat)^2
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 4-parameter spatial-only list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [3] log(nugget)
    ]

    # --- Print Job Info ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
    
    # --- 💥 REVISED: Instantiate the LBFGS Class ---
    model_instance = kernels_repar_space.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 REVISED: Call the LBFGS Optimizer Method ---
    # L-BFGS uses a different set of parameters and doesn't return a scheduler.
    # We pass the list of parameters directly, which set_optimizer will wrap.
    optimizer = model_instance.set_optimizer(
            params_list,     
            lr=1.0,            # L-BFGS often uses lr=1.0 for its line search
            max_iter=20        # Max iterations *per step*
        )

    # --- 💥 REVISED: Call the LBFGS Fit Method ---
    # This method takes 'max_steps' (epochs) and no scheduler.
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_SPATIAL_log_reparam, # Pass the spatial-only method
            max_steps=50 # Total number of L-BFGS steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} steps.")
    print(f"Day {day_idx+1} final results: {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1092, smooth: 0.5
mm_cond_number: 8,
initial parameters: 
 [tensor([3.0647], dtype=torch.float64, requires_grad=True), tensor([0.3567], dtype=torch.float64, requires_grad=True), tensor([0.1177], dtype=torch.float64, requires_grad=True), tensor([0.4055], dtype=torch.float64, requires_grad=True)]
--- Starting L-BFGS Optimization ---
--- Step 1/50 / Loss: 1805.816426 ---
  Param 0: Value=2.1645, Grad=0.004399639763393286
  Param 1: Value=-0.1564, Grad=-0.0009435425054716267
  Param 2: Value=0.9048, Grad=0.00042520187227827744
  Param 3: Value=1.6070, Grad=0.003094805306953441
  Max Abs Grad: 4.399640e-03
------------------------------
--- Step 2/50 / Loss: 1764.367801 ---
  Param 0: Value=2.1644, Grad=-2.8513949725462312e-05
  Param 1: Value=-0.1564, Grad=-1.0744111000847434e-05
  Param 2: Value=0.9048, Grad=1.94917955772117e-05
  Param 3: Value=1.6070, Grad=-0.00012132629137895368
  Max Abs Grad: 1.213263e-04
------------------------

# Result

In [13]:
def cal(a):
    day_indices = [0] 
    for day_idx in day_indices:  

        daily_hourly_map = daily_hourly_maps[day_idx]
        daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

        # Convert initial parameters to a list of 1-element tensors
        params_list = [
            torch.tensor([val], dtype=torch.float64, requires_grad=True, device=device_str) for val in a
        ]
        
        # 💡 CRITICAL: Concatenate the list into a single tensor
        # The full_likelihood function expects a single tensor, not a list
        params_tensor = torch.cat(params_list)

        # Assuming 'kernels_repar_space' has your 'fit_vecchia_adams' class
        model_instance = kernels_repar_space.fit_vecchia_adams_fullbatch(
                smooth = v,
                input_map = daily_hourly_map,
                aggregated_data = daily_aggregated_tensor,
                nns_map = nns_map,
                mm_cond_number = mm_cond_number,
                nheads = nheads
            )
        
        # 💡 Pass the single 'params_tensor' and the correct 4-parameter spatial covariance function
        bb = model_instance.full_likelihood(
            params = params_tensor, 
            input_data = daily_aggregated_tensor, 
            y = daily_aggregated_tensor[:,2], 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam
        )
     
        cov_map = model_instance.cov_structure_saver(params_tensor, model_instance.matern_cov_SPATIAL_log_reparam)
        vecchia_nll = model_instance.vecchia_space_fullbatch( # Change this to your chosen Vecchia implementation
            params = params_tensor, 
            covariance_function = model_instance.matern_cov_SPATIAL_log_reparam, 
            cov_map = cov_map # Assuming cov_map is precomputed or computed internally
        )
 
    return bb, vecchia_nll
   

# Your 4 raw log-parameters.  1782
a = [2.5716621321865656, -0.02410968642134296, 0.7694632395603402, 1.4070729200232799] #1770 1771


a = [3.7883614610682113, 1.6406967353519504, 0.9599948811175316, -19.932468678646213] #1877 1893
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

Full Negative Log-Likelihood: (1893.5065584906438, 1830.4889399723534)


In [57]:
def cc(n,nheads,mm):
    print(n**2 , nheads**2 + (n-nheads)*mm**3)
    return n**2 >nheads**2 + (n-nheads)*mm**3

cc(1000,300,8)

1000000 448400


True

Adams full batch epochs 400 lr 0.01 stepsize 300 gamma 0.5 nheads 300

mm 15 n heads 300

In [14]:
a = [2.5666882131690856, 0.004986659069246286, 0.722128565039769, 1.4149096267261256] # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.6666882131690856, 0.004986659069246286, 0.722128565039769, 1.4149096267261256]  
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.4666882131690856, 0.004986659069246286, 0.722128565039769, 1.4149096267261256] 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

Full Negative Log-Likelihood: (1770.7992508631132, 1768.725666068266)
Full Negative Log-Likelihood: (1771.2938204214065, 1769.3853597027107)
Full Negative Log-Likelihood: (1771.615179336761, 1769.3542019566444)


Adams mini batch + nheads

In [35]:
a = [2.566688217287276, 0.00498666378569817, 0.7221285638881244, 1.4149096243301411, 1.5792193466527076] # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

[2.666688217287276, 0.00498666378569817, 0.7221285638881244, 1.4149096243301411, 1.5792193466527076] 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.466688217287276, 0.00498666378569817, 0.7221285638881244, 1.4149096243301411, 1.5792193466527076]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

Full Negative Log-Likelihood: (1770.7992508637444, 1765.7239272012205)
Full Negative Log-Likelihood: (1770.7992508637444, 1765.7239272012205)
Full Negative Log-Likelihood: (1771.615179332603, 1765.9661415578007)


Lbfgs

In [36]:
a = [2.5651585631971776, 0.0034266437145322264, 0.7226444920120846, 1.4157959166893677] # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.6651585631971776, 0.0034266437145322264, 0.7226444920120846, 1.4157959166893677]  
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.4651585631971776, 0.0034266437145322264, 0.7226444920120846, 1.4157959166893677]
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")


Full Negative Log-Likelihood: (1770.7991513942147, 1765.7310866395708)
Full Negative Log-Likelihood: (1771.2895388942902, 1766.7256224549485)
Full Negative Log-Likelihood: (1771.617359042566, 1765.9762456013634)


# Test2 (smaller conditioning set)

epochs 400 lr 0.01 stepsize 100 (not 300) gamma 0.5 nheads 300

mm 8 ( not 15) n heads 300

cc(1000,300,8)

Admas (full batch minibatch same but minibatch much slower)

🧮 Interpretable Parameters

sigma_sq (Variance): 10.816

range_lon (Longitude Range): 0.849

range_lat (Latitude Range): 0.570

nugget: 3.869

In [54]:
a = [2.645224120507326, 0.16420349697611572, 0.7965928588052055, 1.3529519008400994] # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.745224120507326, 0.16420349697611572, 0.7965928588052055, 1.3529519008400994]  
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.545224120507326, 0.16420349697611572, 0.7965928588052055, 1.3529519008400994] 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")



Full Negative Log-Likelihood: (1771.1122365147507, 1766.1245361855401)
Full Negative Log-Likelihood: (1771.5835939954143, 1767.3549632955146)
Full Negative Log-Likelihood: (1772.1117240453366, 1766.1935232053738)


LBFGS

In [55]:
a = [2.164449396997609, -0.15636298843602242, 0.9048191515471612, 1.6069876322585626] # estimates
bb, nll_value = cal(a)
print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.264449396997609, -0.15636298843602242, 0.9048191515471612, 1.6069876322585626]  
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

a = [2.064449396997609, -0.15636298843602242, 0.9048191515471612, 1.6069876322585626] 
bb, nll_value = cal(a)

print(f"Full Negative Log-Likelihood: {bb.item(), nll_value.item()}")

Full Negative Log-Likelihood: (1774.0437082643498, 1764.3678011661716)
Full Negative Log-Likelihood: (1772.8773105440666, 1764.7327064823778)
Full Negative Log-Likelihood: (1776.1781893155423, 1764.7070853105356)
